# Install requirements

In [ ]:
%pip install tensorflow_addons numpy pandas tensorflow sklearn nltk spacy textblob gensim scipy seaborn matplotlib minio mlflow wordcloud boto3

In [ ]:
import os
cwd = os.getcwd()
import sys
# sys.path is a list of absolute path strings
sys.path.append('/opt/app-root/src/NLP-Openshift-OpenDataHub/')
from src.loadingdata.read_dataset import ReadData
from src.features.build_features import BuildFeatures
from src.modules.build_model import BuildModel
from src.modules.train_model import MLflow, TrainModel
from src.modules.predict_model import Predictor,Transformer,DownloadArtifact


import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.feature_extraction.text as text
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from textblob import TextBlob
from nltk.stem import PorterStemmer,SnowballStemmer
from textblob import Word
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
# from wordcloud import WordCloudfrom wordcloud import WordCloud
from io import StringIO
import string
import gensim
from gensim.models import Word2Vec
import itertools
import scipy
from scipy import spatial
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import joblib

import mlflow
import warnings

from minio import Minio
import subprocess
import ipynbname
warnings.filterwarnings("ignore")
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)


tokenizer = ToktokTokenizer()

# Initialize the config file for mlflow and Minio

In [ ]:

HOST = "http://mlflow:5500"

PROJECT_NAME = "NlpTc"
EXPERIMENT_NAME = "NlpLstm"

os.environ['MLFLOW_S3_ENDPOINT_URL']='http://minio-ml-workshop:9000'
os.environ['AWS_ACCESS_KEY_ID']='minio'
os.environ['AWS_SECRET_ACCESS_KEY']='minio123'
os.environ['AWS_REGION']='us-east-1'
os.environ['AWS_BUCKET_NAME']='raw-data-saeed'



## Define a Function to read from Minio S3 Bucket

In [ ]:
def get_s3_server():
    minioClient = Minio('minio-ml-workshop:9000',
                    access_key='minio',
                    secret_key='minio123',
                    secure=False)

    return minioClient
client = get_s3_server()

## SetUp MLFlow to track the model

In [ ]:
mlflow = MLflow(mlflow, HOST,EXPERIMENT_NAME).SetUp_Mlflow()

# Readinng the data


In [ ]:
train_data, test_data, train_labels, test_labels,enc = ReadData(S3BucketName = "raw-data-saeed",FILE_NAME="data.csv").ReadDataFrameData()

In [ ]:
joblib.dump(enc, 'labelencoder.pkl')  

In [ ]:
# from src.features.build_features import BuildFeatures
train_data.shape

test_labels.shape


# Prepare data for modeling

In [ ]:
BFCLASS = BuildFeatures(TRAIN_DATA=train_data,TEST_DATA=test_data,TRAIN_LABELS=train_labels,TEST_LABELS=test_labels, GloveData="glove.6B.50d.txt",EMBEDDING_DIM=50, WEIGHT_FLAG = False,MLFLOW_S3_ENDPOINT_URL = "minio-ml-workshop:9000",AWS_ACCESS_KEY_ID='minio',AWS_SECRET_ACCESS_KEY = 'minio123',SECURE = False)

In [ ]:
train_data,test_data,train_labels , test_labels,word_index,tokenizer,MAX_SEQUENCE_LENGTH = BFCLASS.PreProcessingTextData()

In [ ]:
joblib.dump(tokenizer, 'tokenizer.pkl')  

# Deep Learning define, train and test model


## Define the Model

In [ ]:
model = BuildModel(WORD_INDEX=word_index, MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH, EMWEIGHTS=[]).SetupModel()

## Train the Model

In [ ]:
model, history= TrainModel(model, tokenizer, enc,train_data, train_labels,test_data, test_labels,HOST, EXPERIMENT_NAME, BATCH_SIZE=64,EPOCHS=2).ModelTraining()

### Plot the training and testing Loss

In [ ]:
fig1 = plt.figure()
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves :RNN - LSTM',fontsize=16)
plt.show()

#### Plot the training and testing Accuracy

In [ ]:
fig1 = plt.figure()
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training acc', 'Validation acc'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves :RNN - LSTM',fontsize=16)
plt.show()

## Test the model

In [ ]:
#predictions on test data
predicted=model.predict(test_data)
predicted

In [ ]:
#model evaluation
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels, predicted.round())
print('precision: \n{}'.format(precision))
print('recall: \n{}'.format(recall))
print('fscore: \n{}'.format(fscore))
print('support: \n{}'.format(support))
print("############################")

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(test_labels.argmax(axis=1), predicted.argmax(axis=1))
fig, ax = plt.subplots(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="BuPu",xticklabels=enc.classes_,yticklabels=enc.classes_)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:

from json import dumps,loads

headers = {"Content-Type" : "application/json"}
testData = {"data":
          {
                "names":
            [
              "consumer complain"
            ],
      "ndarray": ["could longer pay enormous charge hired company nl take either nothing pay day loan company accept term get several letter week threatened take civil action get check"]

          }
        }
testData = dumps(testData)